In [1]:
cd test_resources

/home/clyde/Dropbox/Project Stuff/Notebooks/test_cases/test_resources


In [8]:
from IPython.html import widgets 
from IPython.display import display
from ase.io import read
from ase import Atoms
from gausspy import Gaussian

In [3]:
class Basic_Gaus_Widg():
    def __init__(self, atoms, init_display=True):        
        self.label_widg = widgets.Text(description="Label") 
        
        self.base_method_widg = widgets.DropdownWidget(description="Method", values=["HF", "DFT", "PM6"], value="HF")
        self.dft_method_widg = widgets.DropdownWidget(description="      ", values=["LDA", "B3LYP", "PBE"], value="LDA", visible=False)
        self.method_widg = widgets.ContainerWidget(children=[self.base_method_widg, self.dft_method_widg])
        
        self.basis_widg = widgets.DropdownWidget(description="Basis", values=["STO-3G", "6-31G", "6-31G*"], value="STO-3G")
        self.structure_widg = widgets.Text(description="Structure")
        
        self.input_widg = widgets.ContainerWidget(children = [self.label_widg, self.method_widg, self.basis_widg, self.structure_widg])
        
        self.label_widg.on_trait_change(self.on_label_selection, 'value')
        self.base_method_widg.on_trait_change(self.on_method_selection, 'value')
        self.dft_method_widg.on_trait_change(self.on_dft_selection, 'value')
        self.basis_widg.on_trait_change(self.on_basis_selection, 'value')
        self.structure_widg.on_trait_change(self.on_structure_selection, 'value')                
 
        self.atoms = atoms
        
        self.atoms.calc.label = self.label_widg.value
        self.atoms.calc.method = self.base_method_widg.value
        self.atoms.calc.basis = self.basis_widg.value
        
        if init_display:
            self.display()
        
    def on_label_selection(self):
        self.atoms.calc.label = self.label_widg.value
    
    def on_method_selection(self, name, method_selected):
        if method_selected == 'DFT':
            self.dft_method_widg.visible=True
            self.atoms.calc.method = self.dft_method_widg.value
        else:
            self.dft_method_widg.visible=False
            self.atoms.calc.method = method_selected

    def on_dft_selection(self):
        self.atoms.calc.method = self.dft_method_widg.value
    
    def on_basis_selection(self):
        self.atoms.calc.basis = self.basis_widg.value
        
    def on_structure_selection(self, name, structure_fn):
        if structure_fn:
            try:
                atoms = read(structure_fn)
                if self.atoms.positions != atoms.positions:
                    self.atoms += atoms
            except IOError:
                pass

    def display(self):
        display(self.input_widg)

class Job_Gaus_Widg():
    def __init__(self, atoms, init_display=True):
        #default allocation not fully implemented in IPython yet
        self.time_widg = widgets.IntSliderWidget(description="Time",min=1,max=72,default=1)
        self.time_widg.value=1
        self.nodes_widg = widgets.IntSliderWidget(description="Nodes", min=1, max=16, default=1)
        self.nodes_widg.value=1
        self.memory_widg = widgets.BoundedIntText(description="Memory", min=400, max=2000, default=400)
        self.memory_widg.value=400
        
        self.queue_widg = widgets.DropdownWidget(description="Queue", values=["PQChem", "PQMB", "Default"], value="Default")
        self.version_widg = widgets.DropdownWidget(description="Version", values=["G09", "GDV", "Local G09"], value="G09")
        self.job_widg = widgets.ContainerWidget(children=[self.time_widg, self.nodes_widg, self.memory_widg, self.queue_widg, self.version_widg])
        
        self.time_widg.on_trait_change(self.on_time_selection, 'value')
        self.nodes_widg.on_trait_change(self.on_nodes_selection, 'value')
        self.memory_widg.on_trait_change(self.on_memory_selection, 'value')
        self.queue_widg.on_trait_change(self.on_queue_selection, 'value')
        self.version_widg.on_trait_change(self.on_version_selection, 'value')
        
        self.atoms = atoms
        
        if init_display:
            display(self.job_widg)
        
    def on_time_selection(self):
        self.atoms.calc.job_params['time'] = self.time_widg.value

    def on_nodes_selection(self):
        self.atoms.calc.job_params['nodes'] = self.nodes_widg.value

    def on_memory_selection(self):
        self.atoms.calc.job_params['memory'] = self.memory_widg.value * self.nodes_widg.value

    def on_queue_selection(self):
        self.atoms.calc.job_params['queue'] = self.queue_widg.value

    def on_version_selection(self):
        self.atoms.calc.job_params['version'] = self.version_widg.value

    def display(self):
        display(self.job_widg)
        
class Gauss_Inp(object):
    def __init__(self):
        self.atoms = Atoms()
        self.atoms.set_calculator(Gaussian())
        
        self.basic_input = Basic_Gaus_Widg(self.atoms, init_display=False)
        self.job_input = Job_Gaus_Widg(self.atoms, init_display=False)
        
        self.basic_input.display()
        
    #access atoms object via composition
    def __getattr__(self, name):
        return self.atoms.__getattribute__(name)
    
        
    #allows tab completion but too many accessible variables!    
    #def __dir__(self):
    #    return self.__dict__.keys() + dir(self.atoms)

In [4]:
graph_66 = Gauss_Inp()

KeyError: 'HF'

In [5]:
graph_66.job_input.display()

NameError: name 'graph_66' is not defined

In [38]:
IPython.notebook.get_selected_cell().get_text()

NameError: name 'IPython' is not defined

In [6]:
graph_66_v2 = read('6_6_graphene.com')
graph_66_v2.set_calculator(Gaussian(label='test_calc', method='B3LYP', basis='STO-3G'))
graph_66_v2.calc.set_job(nodes=8, memory=8*1200, time=3, version='g09')
#graph_66.calc.start()